# How do I write files to my own _cloud storage_?
### Overview
Connecting to cloud storage facilitates sharing and leveraging bioinformatics data that is commonly stored there (details [here](http://docs.sevenbridges.com/docs/connecting-cloud-storage-overview)). Here, we show an example of **writing** files from the _Seven Bridges Platform_ to a users _Amazon Web Services_ **bucket**. 

### Prerequisites
 1. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 2. You need to have an AWS account with an S3 bucket
 3. You need to be a member (or owner) of a project that has some files in it, e.g. if you followed this [recipe](files_copyFromPublicReference.ipynb)
 
### Setting up Identity and Access Management (IAM) access 
There is a detailed [tutorial](http://docs.sevenbridges.com/v1.0/docs/aws-cloud-storage-tutorial) to set up access to your buckets. Please follow that tutorials and make note of your **policy name(s)**. Also make sure to generate an _secret access key_.

## Imports
We import the _Api_ class from the official sevenbridges-python bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The _Api_ object needs to know your **auth\_token** and the correct path. Here we assume you are using the .sbgrc file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify platform {cgc, sbg}
prof = 'sbpla'


config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Mount the volume
Since we have configured the IAM policy in the prior step, we can now mount the volume. Make sure to **replace** (BUCKET-NAME, INSERT ACCESS KEY ID, and INSERT SECRET ACCESS KEY) below

### Warning
You only need to mount a volume **once**. If you have already done this, then skip this cell and go to _Get details of the volume_

#### Additional information
 * Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/create-a-volume-v2)

In [ ]:
volume_import = api.volumes.create_s3_volume(
    name='my_volume_RW', 
    bucket='BUCKET-NAME',
    access_key_id='INSERT ACCESS KEY ID',
    secret_access_key = 'INSERT SECRET ACCESS KEY',
    access_mode='RW')

## Get details of the volume
Let's list all our volumes and get one volume's details to make sure it's mounted as we expected.

### Warning
We've hardcoded the single_volume to detail, this could easily be selected by matching 'name' instead

#### Additional information
 * Detailed documentation of these particular REST architectural style request are available [here (list)](http://docs.sevenbridges.com/docs/list-volumes-v2) and [here (details)](http://docs.sevenbridges.com/docs/get-details-of-a-volume-v2)

In [ ]:
# [USER INPUT] Set volume name here:
volume_name = 'my_volume_RW'  


all_volumes = api.volumes.query()

my_volume = [v for v in all_volumes.all() 
            if v.name == volume_name]

if not my_volume:
    print('Volume {} does not exist, check name / mounting'
          .format(volume_name))
    raise KeyboardInterrupt
else:
    my_volume = my_volume[0]

print(my_volume.name)
print(dir(my_volume))

## Get some files to move
Here we pick a project and a few files to move. In case you don't have any files, you could add them via the [GUI](http://docs.sevenbridges.com/docs/copy-files-using-the-visual-interface) or an API [recipe](files_copyFromPublicReference.ipynb). 

#### PROTIPS
 * A detailed _recipe_ for listing projects is [here](projects_listAll.ipynb)
 * A detailed _recipe_ for listing files is [here](files_listAll.ipynb)

In [ ]:
# [USER INPUT] Set project name here:
project_name = 'Azzurri'
extension = 'txt'


# Find your project
my_project = [p for p in api.projects.query(limit=100).all() \
             if p.name == project_name]

if not my_project:    # exploit fact that empty list is False, {list, tuple, etc} is True
    print('No project named (%s) found, please check spelling' % project_name)
    raise KeyboardInterrupt
else:
    my_project = my_project[0]
    
# List files in that project matching your extension
my_files = api.files.query(limit=100, project=my_project)
selected_files = [f for f in my_files.all() \
                  if f.name[-len(extension):] == extension]

## Start an export job
Spin up a job to export files from the _Seven Bridges Platform_ to your _Cloud Storage_ (Volume).

#### Additional information
 * Detailed documentation of these particular REST architectural style request are available [here](http://docs.sevenbridges.com/docs/start-an-export-job-v2)

In [ ]:
# Loop through selected files, start one job for each.
exports = []
for f in selected_files:
    export = api.exports.submit_export(
        file=f,                        
        volume=my_volume,                     
        location=f.name)
    
    exports.append(export)

    print("File {} queued for export to your cloud storage \n"
          .format(f.name))

## List and detail export jobs
The first call (list) is sufficient to keep track of the job progress in 

```python
    my_volume.get_imports(project=my_project)
```

#### Additional information
 * Detailed documentation of these particular REST architectural style request are available [here](http://docs.sevenbridges.com/docs/list-export-jobs-v2) and [here](http://docs.sevenbridges.com/docs/get-details-of-an-export-job-v2)

In [ ]:
all_export_jobs = my_volume.get_exports(project=my_project)

for j in all_export_jobs:
    print('File {} import is {}'
         .format(j.destination, j.state))
print('\n')